In [2]:
import pandas as pd
import sqlite3
import datetime as dt


In [3]:
# Use the same dataset as segment 5 for this question. Connect to your sqlite3 database with
data = sqlite3.connect('db.sqlite3')
# Python and export your data into a pandas DataFrame.
df = pd.read_sql_query("SELECT * FROM sales", data)

df.sale_date = pd.to_datetime(df.sale_date)
df.store_name = df.store_name.astype(str)

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10950 entries, 0 to 10949
Data columns (total 3 columns):
store_name    10950 non-null object
sale_date     10950 non-null datetime64[ns]
num_sold      10858 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 213.9+ KB


,store_name,sale_date,num_sold
0,S1,2017-01-01,76.0
1,S1,2017-01-02,64.0
2,S1,2017-01-03,53.0
3,S1,2017-01-04,65.0
4,S1,2017-01-05,58.0


In [78]:
# Write a pandas program ( hw1/q6/main.py ) that outputs the answers to the following queries,
# one per line:
# 6.1 Which store had the highest mean sale in 2017?
## take the average of all sales per day for each store

##df.groupby(df.store_name).mean().sort_values(by='num_sold', ascending=False)
##pd.strftime("%Y", df.sale_date)
##df.sale_date.dt.strftime("%Y")
df_17 = df[(df.sale_date>='2017') & (df.sale_date<'2018')]
df_17.groupby(df_17.store_name).mean().sort_values(by='num_sold', ascending=False)



,num_sold
store_name,
S2,75.561983
S6,67.268493
S1,62.138504
S7,57.154930
S3,52.367403
S4,52.367403
S10,50.190083
S5,49.219178
S9,47.810056


In [41]:
# 6.2 Which day showed the highest variance in sales across different stores?
##calculate variance in S1 thru S10 for a single day
df[df.sale_date == '2017-01-01'].var()

start_date = df.sale_date.sort_values()[0].date()
end_date = df.sale_date.sort_values().iloc[-1].date()
daterange = pd.date_range(start_date, end_date)
df_new = pd.DataFrame(columns=[])
variances = list()

for single_date in daterange:
    ##df_new.append(pd.Series(df[df.sale_date == single_date].num_sold.var()),ignore_index=True)
    ##print(df[df.sale_date == single_date].num_sold.var())
    #variances.append( df[df.sale_date == single_date].num_sold.var() )
    ##you'll have to make a df with the sale date AND the variance for that sale date
    variances.append( ( single_date, df[df.sale_date == single_date].num_sold.var()) )
    ##df_new.append(pd.Series(df[df.sale_date == single_date].num_sold.var()),ignore_index=True)
#max(variances)
max(variances, key=lambda item:item[1])

## 2017-03-16 had the highest variance in sales across all stores


(Timestamp('2017-03-16 00:00:00', freq='D'), 487.9555555555556)

In [39]:
# 6.3 Which year showed the highest median sale for the store S5?
# df[df.store_name == 'S5']
# start_year = dt.datetime(2017,1,1)
# end_year = dt.datetime(2020,1,1)

# yearrange = pd.date_range(start_year,end_year, freq = 'Y')
# #yearrange
# for single_year in yearrange:
#     print(df.sale_date[df.sale_date.year == single_year].median())

#make a new column from just the years, add it to DF
df.loc[:,'year'] = df.loc[:,'sale_date'].dt.year

print(df.loc[df.store_name =='S5',:].groupby('year')['num_sold']
                            .median()
                            .reset_index()
                            .sort_values('num_sold',ascending=False)
                            .head(1))





   year  num_sold
2  2019      47.0


In [105]:
# 6.4 Which store recorded the highest number of sales for the largest number of days?

##select which store "won" on a single day
df.loc[df.sale_date == pd.Timestamp(start_date),:].sort_values('num_sold', ascending=False).head(1)
                                                
##iterate over all thedays
start_date = df.sale_date.sort_values()[0].date()
end_date = df.sale_date.sort_values().iloc[-1].date()
daterange = pd.date_range(start_date, end_date)
day_winner = pd.DataFrame()

for single_date in daterange:
    day_winner = day_winner.append( df.loc[df.sale_date == pd.Timestamp(single_date),:].sort_values('num_sold', ascending=False).head(1) )
#.reset_index()
#.sort_values('num_sold', ascending=False)
day_winner.store_name.value_counts().head(1)



S2    607
Name: store_name, dtype: int64

In [133]:
# 6.5 Which store ranks 5th in the cumulative number of units sold over the 3-year interval?
print(df.groupby('store_name')['num_sold'].sum().sort_values(ascending=False)[[4]])



store_name
S4    57086.0
Name: num_sold, dtype: float64


In [135]:
# 6.6 Your program should create a file named repaired.csv in the directory hw1/q4 which
# contains the same data as data.csv , but with “N/A” values replaced with the median
# sale of that store, over the entire 3-year interval. Retain the header row found in
# data.csv.
rep = df.copy()
rep.loc[:,'median_sales'] = rep.groupby('store_name')['num_sold'].transform(lambda x: x.median())
rep[['store_name','median_sales']].drop_duplicates()
rep.loc[:,'num_sold'] = rep['num_sold'].combine_first(rep['median_sales'])
rep = rep.pivot(index='sale_date',columns='store_name',values='num_sold').reset_index()
 
# re-create the data.csv header columns for each store: s1-s10
cols = ['sale_date']
cols.extend(['S'+str(r+1) for r in range(10)])
rep = rep.loc[:, cols]
rep.to_csv('../q6/repaired.csv', index=False)